In [ ]:
using ModelingToolkit
using MethodOfLines
using DifferentialEquations
using DomainSets
using FFTW
using CairoMakie

In [ ]:
xleft::Float64 = 0.0
xright::Float64 = 1.0
t0::Float64 = 0.0
T::Float64 = 70.0
N = 100
order = 2
step = (xright - xleft)/N
grid = collect(xleft:step:xright)

#Definition of constants (placeholders):
gamma::Float64 = 2
gamma_3::Float64 = 50
c::Float64 = 1
omega_d::Float64 = 2*pi*3

#Define parameters and variables (note: u(..) is valid as a symbolic function notation):
@parameters x t
@variables u(..)

#Define differential operators:
Dt = Differential(t)
Dx = Differential(x)
Dtt = Differential(t)^2
Dxx = Differential(x)^2

#Define source function (placeholder constant spatial term):
source_f(x, t) = 1000 * sin(omega_d*t)

#Define initial condition (placeholder sinusoidal):
#u0(x, t) = exp(-(35*pi)*(x-0.5)^2)
u0(x, t) = 0
v0(x) = 0


#Define equation (with and without cubic damping term):
equation = [Dtt(u(x,t)) + gamma*Dt(u(x,t)) + gamma_3*(Dt(u(x,t)))^3 ~ c^2*Dxx(u(x,t)) + source_f(x, t)]
#equation = [Dtt(u(x,t)) ~ c^2*Dxx(u(x,t))]


#Define domains:
domains = [x ∈ Interval(xleft, xright), t ∈ Interval(t0, T)]

#Define boundary conditions:
bound_conditions = [u(x, 0) ~ u0(x, 0),
                     u(xleft, t) ~ 0,
                     u(xright, t) ~ 0,
                     Dt(u(x, 0)) ~ v0(x)]

In [ ]:
#Create PDE system and assign identifier (order matters: eq, bcs, domains, paramers, variables; format parameters and variables as lists, provide arguments whenever they exist):
@named pde_sys = PDESystem(equation, bound_conditions, domains, [x, t], [u(x,t)])

#Define a discretization (first argument specifies parameters to discretize, second indicates parameters to keep continuous, third provides the approximation order):
discretization = MOLFiniteDifference([x=>grid], t, approx_order=order)

#Proceed with discretization:
@time eq_discretized = discretize(pde_sys,discretization)

#Solve the resulting system (specified solver dictates whether or not time marching is used, as well as the method: Euler(), ImplicitEuler(), or Trapezoidal(), all with given time steps):
#Warning: if the time step is not explicitly given, solver may default to actual method of lines, instead of time marching\
dt = 1e-2
solution= solve(eq_discretized, ImplicitEuler(), dt=dt, saveat=0.01)

In [ ]:
#Plot results:
discrete_t = solution.t

sol_u = solution[u(x, t)]

Nx = length(grid)
Nt = length(solution.t)


In [ ]:
# Fourier Transform analysis

k = 3900
fs = N

u_at_k = sol_u[1:end, k]

F = fftshift(fft(u_at_k))
freqs =  fftshift(fftfreq(length(u_at_k), fs))

# Plotting with Makie
f = Figure()
ax1 = Axis(f[1, 1],
    xlabel = "Distance (m)",
    ylabel = "Displacement (m)",
)

ax2 = Axis(f[1, 2], xlabel = "Frequency (Hz)",
    ylabel = "Amplitude (m)",
)

lines!(ax1, grid, u_at_k)
lines!(ax2, freqs, abs.(F) / fs)
xlims!(ax2, -50, 50)

f

In [ ]:
u_at_point = sol_u[36, 6500:end]
fs_time = Nt/T

F = fftshift(fft(u_at_point))
freqs =  fftshift(fftfreq(length(u_at_point), fs_time))

# Plotting with Makie
f = Figure()
ax1 = Axis(f[1, 2], xlabel = "Frequency (Hz)",
    ylabel = "Amplitude (m)",
)

lines!(ax1, freqs, abs.(F) / fs_time)
xlims!(ax1, -50, 50)

f

###### 